### PREPROCESAMIENTO

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle

In [5]:
#Cargar el csv
df = pd.read_csv("../bin/data_preprocess.csv")

In [6]:
df

,id_provincia,kilometraje,id_concesionario,id_distintivo_ambiental,garantia,precio_contado,precio_nuevo,largo,ancho,alto,...,par,velocidad_max,aceleracion,mes_matriculacion,ano_matriculacion,co2,num_cilindros,id_sobrealimentacion,id_marca,id_modelo
0,Madrid,58000.0,RUTA 66,C,12.0,19990.0,37250.0,4633.0,1811.0,1429.0,...,320.0,215.0,8.7,11,2018,118.0,4.0,Turbo de geometría variable,BMW,SERIE 3
1,Madrid,34680.0,RUTA 66,ECO,12.0,12990.0,19300.0,3571.0,1627.0,1488.0,...,92.0,167.0,13.8,12,2021,NaN,3.0,Turbo,FIAT,500
2,Madrid,66933.0,AUTOMOTOR DURSAN,C,12.0,17390.0,26650.0,4377.0,1806.0,1590.0,...,260.0,182.0,11.9,4,2017,103.0,4.0,Turbo,NISSAN,QASHQAI
3,Sevilla,159000.0,Mules Car,C,12.0,8000.0,22425.0,4702.0,1809.0,1499.0,...,300.0,195.0,11.4,1,2017,99.0,4.0,Turbo,OPEL,ASTRA
4,Madrid,62695.0,CLICARS MADRID,0 EMISIONES,12.0,22990.0,44700.0,4545.0,1805.0,1685.0,...,193.0,162.0,10.9,7,2022,NaN,4.0,Turbo,MITSUBISHI,ECLIPSE CROSS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34057,Alicante,28000.0,BMW HISPAMOVIL,ECO,24.0,79500.0,82000.0,4713.0,1897.0,1676.0,...,700.0,250.0,4.9,12,2023,NaN,6.0,Turbo de geometría variable,BMW,X3
34058,Murcia,12280.0,AUDI HUERTAS MOTOR MURCIA,C,12.0,24000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8,2023,NaN,NaN,NaN,AUDI,A1
34059,Madrid,18744.0,STAR MADRID,0 EMISIONES,36.0,38990.0,57005.0,4684.0,1834.0,1667.0,...,375.0,160.0,9.2,8,2023,NaN,NaN,NaN,MERCEDES-BENZ,EQB
34060,Madrid,101353.0,CLICARS MADRID,C,12.0,13490.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8,2018,NaN,NaN,NaN,RENAULT,SCENIC


In [10]:
#Calcular en porcentaje de nulos
porcentaje_nan = df.isna().mean() * 100
porcentaje_nan

id_provincia                0.020551
kilometraje                 0.184957
id_concesionario            0.000000
id_distintivo_ambiental     2.427926
garantia                    2.060948
precio_contado              6.262110
precio_nuevo               28.594915
largo                      29.549058
ancho                      29.505020
alto                       29.572544
capacidad_maletero         30.001174
num_plazas                 29.493277
batalla                    29.546122
peso                       24.194117
num_puertas                 0.989372
consumo_medio              48.951911
consumo_carretera          53.922259
consumo_urbano             53.913452
deposito                   35.535201
combustible                 1.006987
cilindrada                 34.225823
tipo_cambio                 0.000000
id_traccion                29.493277
num_marchas                26.243321
potencia_kw                29.408138
potencia_cv                 0.974693
par                        29.930715
v

### IMPUTACION DE DATOS

In [12]:
#Funcion Imputacion de datos

def imputar_datos(df):

    #Separamos columnas numericas de categoricas
    columnas_numericas = df.select_dtypes(include=['float', 'int']).columns
    columnas_categoricas = df.select_dtypes(include=['object']).columns
    
    #Inicializar Knn para las columnas numericas
    knn_imputer = KNNImputer(n_neighbors=3)
    
    #Aplicar knn imputer a las columnas numericas
    df_numericas = df[columnas_numericas]
    df_imputado_numericas = knn_imputer.fit_transform(df_numericas)
    
    #Convertir los datos imputados a df
    df_imputado_numericas = pd.DataFrame(df_imputado_numericas, columns=columnas_numericas)
    
    #Inicializar SimpleImputer para las columnas categoricas
    simple_imputer = SimpleImputer(strategy='most_frequent')
    
    #Aplicar SimpleImputer para las columnas categoricas
    df_categoricas = df[columnas_categoricas]
    df_imputado_categoricas = simple_imputer.fit_transform(df_categoricas)
    
    #Convertir las columnas categoricas imputadas a df
    df_imputado_categoricas = pd.DataFrame(df_imputado_categoricas,columns=columnas_categoricas)
    
    #Combinamos los datos imputados
    df_imputado = pd.concat([df_imputado_numericas, df_imputado_categoricas], axis =1)
    
    #verificamos que no hay valores nulos
    print(df_imputado.isnull().sum())
    
    return df_imputado

In [14]:
#Imputacion de datos
df_imputado = imputar_datos(df)

df_imputado
#Guardamos el df si es necesario
#df_imputado.to_csv('Coches-segunda-mano/bin/df_imputado.csv', index=False)

kilometraje                0
garantia                   0
precio_contado             0
precio_nuevo               0
largo                      0
ancho                      0
alto                       0
capacidad_maletero         0
num_plazas                 0
batalla                    0
peso                       0
num_puertas                0
consumo_medio              0
consumo_carretera          0
consumo_urbano             0
deposito                   0
cilindrada                 0
num_marchas                0
potencia_kw                0
potencia_cv                0
par                        0
velocidad_max              0
aceleracion                0
mes_matriculacion          0
ano_matriculacion          0
co2                        0
num_cilindros              0
id_provincia               0
id_concesionario           0
id_distintivo_ambiental    0
combustible                0
tipo_cambio                0
id_traccion                0
id_sobrealimentacion       0
id_marca      

,kilometraje,garantia,precio_contado,precio_nuevo,largo,ancho,alto,capacidad_maletero,num_plazas,batalla,...,num_cilindros,id_provincia,id_concesionario,id_distintivo_ambiental,combustible,tipo_cambio,id_traccion,id_sobrealimentacion,id_marca,id_modelo
0,58000.0,12.0,19990.0,37250.000000,4633.000000,1811.000000,1429.000000,480.000000,5.0,2810.000000,...,4.000000,Madrid,RUTA 66,C,Diesel,Manual,trasera,Turbo de geometría variable,BMW,SERIE 3
1,34680.0,12.0,12990.0,19300.000000,3571.000000,1627.000000,1488.000000,185.000000,4.0,2300.000000,...,3.000000,Madrid,RUTA 66,ECO,Gasolina,Manual,delantera,Turbo,FIAT,500
2,66933.0,12.0,17390.0,26650.000000,4377.000000,1806.000000,1590.000000,430.000000,5.0,2646.000000,...,4.000000,Madrid,AUTOMOTOR DURSAN,C,Diesel,Manual,delantera,Turbo,NISSAN,QASHQAI
3,159000.0,12.0,8000.0,22425.000000,4702.000000,1809.000000,1499.000000,540.000000,5.0,2662.000000,...,4.000000,Sevilla,Mules Car,C,Diesel,Manual,delantera,Turbo,OPEL,ASTRA
4,62695.0,12.0,22990.0,44700.000000,4545.000000,1805.000000,1685.000000,404.000000,5.0,2670.000000,...,4.000000,Madrid,CLICARS MADRID,0 EMISIONES,Híbrido Enchufable,Automático,total permanente,Turbo,MITSUBISHI,ECLIPSE CROSS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34057,28000.0,24.0,79500.0,82000.000000,4713.000000,1897.000000,1676.000000,550.000000,5.0,2864.000000,...,6.000000,Alicante,BMW HISPAMOVIL,ECO,Diesel,Automático,total permanente,Turbo de geometría variable,BMW,X3
34058,12280.0,12.0,24000.0,47729.666667,4497.333333,1829.666667,1509.666667,443.000000,5.0,2713.333333,...,3.333333,Murcia,AUDI HUERTAS MOTOR MURCIA,C,Gasolina,Manual,delantera,Turbo,AUDI,A1
34059,18744.0,36.0,38990.0,57005.000000,4684.000000,1834.000000,1667.000000,495.000000,5.0,2829.000000,...,4.000000,Madrid,STAR MADRID,0 EMISIONES,Eléctrico,Automático,delantera,Turbo,MERCEDES-BENZ,EQB
34060,101353.0,12.0,13490.0,27806.666667,4320.333333,1776.666667,1477.000000,399.666667,5.0,2622.666667,...,3.333333,Madrid,CLICARS MADRID,C,Gasolina,Manual,delantera,Turbo,RENAULT,SCENIC


### TRANSFORMACION MATEMATICA

In [ ]:
#Analizamos la distribucion actual
#Ejecutr para ver los graficos
for col in columnas_numericas:
    plt.figure(figsize=(6, 4))
    plt.hist(df_imputado[col], bins=30, alpha=0.7, color='blue', edgecolor='black')
    plt.title(f"Distribución de {col}")
    plt.xlabel(col)
    plt.ylabel("Frecuencia")
    plt.show()

In [28]:
#Funcion para aplicar las transformaciones
def transformar_columnas_numericas(df):
    
    # Copiar el DataFrame original para preservar los datos originales
    df_transformado = df.copy()
    
    # Identificar columnas numéricas continuas
    columnas_numericas = df_transformado.select_dtypes(include=['float64', 'int64']).columns
    
    for col in columnas_numericas:
        if (df_transformado[col] > 0).all():  # Comprobar si todos los valores son positivos
            # Aplicar logaritmo
            df_transformado[f"{col}_log"] = np.log(df_transformado[col])
            # Aplicar raíz cuadrada
            df_transformado[f"{col}_sqrt"] = np.sqrt(df_transformado[col])
        else:
            print(f"Columna '{col}' contiene valores no positivos. Transformaciones no aplicadas.")

    return df_transformado

In [ ]:
# Aplicar las transformaciones
df_transformado = transformar_columnas_numericas(df_imputado)

In [34]:
df_transformado

,kilometraje,garantia,precio_contado,precio_nuevo,largo,ancho,alto,capacidad_maletero,num_plazas,batalla,...,ano_matriculacion_sqrt_log,ano_matriculacion_sqrt_sqrt,co2_log_log,co2_log_sqrt,co2_sqrt_log,co2_sqrt_sqrt,num_cilindros_log_log,num_cilindros_log_sqrt,num_cilindros_sqrt_log,num_cilindros_sqrt_sqrt
0,58000.0,12.0,19990.0,37250.000000,4633.000000,1811.000000,1429.000000,480.000000,5.0,2810.000000,...,3.804931,6.702399,1.562490,2.184190,2.385342,3.295873,0.326634,1.177410,0.693147,1.414214
1,34680.0,12.0,12990.0,19300.000000,3571.000000,1627.000000,1488.000000,185.000000,4.0,2300.000000,...,3.805674,6.704889,1.539753,2.159500,2.331720,3.208680,0.094048,1.048147,0.549306,1.316074
2,66933.0,12.0,17390.0,26650.000000,4377.000000,1806.000000,1590.000000,430.000000,5.0,2646.000000,...,3.804683,6.701569,1.533578,2.152842,2.317364,3.185733,0.326634,1.177410,0.693147,1.414214
3,159000.0,12.0,8000.0,22425.000000,4702.000000,1809.000000,1499.000000,540.000000,5.0,2662.000000,...,3.804683,6.701569,1.524995,2.143623,2.297560,3.154342,0.326634,1.177410,0.693147,1.414214
4,62695.0,12.0,22990.0,44700.000000,4545.000000,1805.000000,1685.000000,404.000000,5.0,2670.000000,...,3.805921,6.705718,1.536349,2.155827,2.323795,3.195993,0.326634,1.177410,0.693147,1.414214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34057,28000.0,24.0,79500.0,82000.000000,4713.000000,1897.000000,1676.000000,550.000000,5.0,2864.000000,...,3.806168,6.706547,1.611119,2.237948,2.504205,3.497690,0.583198,1.338566,0.895880,1.565085
34058,12280.0,12.0,24000.0,47729.666667,4497.333333,1829.666667,1509.666667,443.000000,5.0,2713.333333,...,3.806168,6.706547,1.561302,2.182892,2.382509,3.291208,0.185627,1.097257,0.601986,1.351200
34059,18744.0,36.0,38990.0,57005.000000,4684.000000,1834.000000,1667.000000,495.000000,5.0,2829.000000,...,3.806168,6.706547,1.536349,2.155827,2.323795,3.195993,0.326634,1.177410,0.693147,1.414214
34060,101353.0,12.0,13490.0,27806.666667,4320.333333,1776.666667,1477.000000,399.666667,5.0,2622.666667,...,3.804931,6.702399,1.560105,2.181587,2.379660,3.286523,0.185627,1.097257,0.601986,1.351200


In [ ]:
#Guardar el DataFrame con las columnas transformadas
# df.to_csv('/Coches-segunda-mano/bin/data_transformed.csv', index=False)
# print("Transformaciones aplicadas y guardadas en 'data_transformed.csv'")

### ENCODING

In [37]:
import os
import pickle
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

#Aplica los encoders y los guarda en archivos pickle.
def aplicar_encoders(df, ruta_encoders='encoders/'):
    
    os.makedirs(ruta_encoders, exist_ok=True)  # Crear la carpeta si no existe
    df_transformado = df.copy()  # Crear una copia para trabajar sin modificar el original

    # Detectar columnas categóricas
    columnas_categoricas = df.select_dtypes(include=['object', 'category']).columns.tolist()
    
    for col in columnas_categoricas:
        print(f"Procesando columna: {col}")
        
        # Aplicar LabelEncoder
        encoder = LabelEncoder()
        df_transformado[col] = encoder.fit_transform(df_transformado[col])
        
        # Guardar el encoder en un archivo pickle
        with open(os.path.join(ruta_encoders, f"{col}_encoder.pickle"), "wb") as file:
            pickle.dump(encoder, file)
    
    print(f"Encoders aplicados y guardados en: {ruta_encoders}")
    return df_transformado

In [39]:
# Aplicar los encoders y guardar los archivos pickle
df_codificado = aplicar_encoders(df, ruta_encoders='encoders/')

Procesando columna: id_provincia
Procesando columna: id_concesionario
Procesando columna: id_distintivo_ambiental
Procesando columna: combustible
Procesando columna: tipo_cambio
Procesando columna: id_traccion
Procesando columna: id_sobrealimentacion
Procesando columna: id_marca
Procesando columna: id_modelo
Encoders aplicados y guardados en: encoders/


### ESCALAR DATOS

In [47]:
#Crear una carpeta para guardar el objeto escalador
os.makedirs("escaladores", exist_ok=True)

In [59]:
#Funcion para escalar los datos y guardar el objeto escalador en un archivo pickle
def escalar_datos(df, metodo="standard", ruta_pickle="escaladores/"):

    # Seleccionar solo las columnas numéricas
    columnas_numericas = df.select_dtypes(include=["float64", "int64"]).columns
    df_numerico = df[columnas_numericas]
    
    # Elegir el escalador
    if metodo == "standard":
        scaler = StandardScaler()
    elif metodo == "minmax":
        scaler = MinMaxScaler()
    else:
        raise ValueError("Método inválido. Usa 'standard' o 'minmax'.")
    
    # Ajustar el escalador y transformar los datos
    datos_escalados = scaler.fit_transform(df_numerico)
    
    # Guardar el escalador en un archivo pickle
    nombre_pickle = f"{metodo}_scaler.pickle"
    ruta_completa = os.path.join(ruta_pickle, nombre_pickle)
    with open(ruta_completa, "wb") as file:
        pickle.dump(scaler, file)
    print(f"Escalador guardado en: {ruta_completa}")
    
    # Crear un nuevo DataFrame con los datos escalados
    df_escalado = pd.DataFrame(datos_escalados, columns=columnas_numericas, index=df.index)
    
    # Añadir las columnas no numéricas al DataFrame final
    columnas_no_numericas = df.select_dtypes(exclude=["float64", "int64"]).columns
    df_final = pd.concat([df_escalado, df[columnas_no_numericas]], axis=1)
    
    return df_final

# Aplicar el escalador al df_codificado
df_escalado = escalar_datos(df_codificado, metodo="standard", ruta_pickle="escaladores")

# Verificar el DataFrame escalado
df_escalado

Escalador guardado en: escaladores\standard_scaler.pickle


,kilometraje,garantia,precio_contado,precio_nuevo,largo,ancho,alto,capacidad_maletero,num_plazas,batalla,...,ano_matriculacion_sqrt,id_provincia,id_concesionario,id_distintivo_ambiental,combustible,tipo_cambio,id_traccion,id_sobrealimentacion,id_marca,id_modelo
0,-0.108328,-0.591458,-0.497130,-0.159194,0.720990,-0.593789,-1.731187,-0.008601,0.091866,0.901972,...,-1.036545,28,595,2,0,1,3,3,7,388
1,-0.567216,-0.591458,-0.785609,-0.989533,-3.497902,-3.209607,-1.131999,-2.502059,-2.922147,-3.355802,...,0.031386,28,595,3,3,1,0,2,16,25
2,0.067455,-0.591458,-0.604280,-0.649534,-0.295994,-0.664871,-0.096116,-0.431221,0.091866,-0.467194,...,-1.392699,28,79,2,0,1,0,2,36,355
3,1.879140,-0.591458,-0.991254,-0.844976,0.995098,-0.622222,-1.020286,0.498543,0.091866,-0.333617,...,-1.392699,36,528,2,0,1,0,2,38,59
4,-0.015940,-0.591458,-0.373497,0.185431,0.371402,-0.679088,0.868678,-0.650983,0.091866,-0.266829,...,0.387187,28,260,0,5,0,2,2,35,185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34057,-0.698664,0.677195,1.955353,1.910871,1.038797,0.628821,0.777276,0.583067,0.091866,1.352795,...,0.742900,1,207,3,0,0,2,3,7,451
34058,-1.008001,-0.591458,-0.331873,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.742900,29,30,2,3,1,4,4,5,39
34059,-0.880803,1.945847,0.285884,0.754641,0.923592,-0.266812,0.685875,0.118185,0.091866,1.060595,...,0.742900,28,625,0,1,0,0,4,32,194
34060,0.744768,-0.591458,-0.765004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.036545,28,260,2,3,1,4,4,41,381


### TRAIN Y TEST

In [ ]:
# Definir los conjuntos de Train y Test.
X = df_escalado.drop('precio', axis=1)  
y = df_escalado['precio']  

# Dividir en Train y Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Si no tienes una columna objetivo y quieres dividir todo el DataFrame
X_train, X_test = train_test_split(df_escalado, test_size=0.2, random_state=42)